In [14]:
from google.cloud import bigquery as bq
import os
import pandas as pd
import util_functions as util

service_credentials = 'Service_Credentials/big-query-horse-play-f37757d450b8.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = service_credentials

bq_client = bq.Client()

In [8]:
sql = """
    SELECT
      CONCAT(
        'https://stackoverflow.com/questions/',
        CAST(id as STRING)) as url,
      view_count
    FROM `bigquery-public-data.stackoverflow.posts_questions`
    WHERE tags like '%google-bigquery%'
    ORDER BY view_count DESC
    LIMIT 10
    """

In [10]:
# Defines the job to run
query_job = bq_client.query(sql)

# Runs it and returns the results
results = query_job.result()

In [13]:
# Need to iterate through the table results
for row in results:
    print(row)

Row(('https://stackoverflow.com/questions/13530967', 44081), {'url': 0, 'view_count': 1})
Row(('https://stackoverflow.com/questions/13221978', 31409), {'url': 0, 'view_count': 1})
Row(('https://stackoverflow.com/questions/22879669', 29177), {'url': 0, 'view_count': 1})
Row(('https://stackoverflow.com/questions/6607552', 26722), {'url': 0, 'view_count': 1})
Row(('https://stackoverflow.com/questions/16609219', 23515), {'url': 0, 'view_count': 1})
Row(('https://stackoverflow.com/questions/22004216', 23226), {'url': 0, 'view_count': 1})
Row(('https://stackoverflow.com/questions/35159967', 21839), {'url': 0, 'view_count': 1})
Row(('https://stackoverflow.com/questions/10604135', 21751), {'url': 0, 'view_count': 1})
Row(('https://stackoverflow.com/questions/10644993', 21110), {'url': 0, 'view_count': 1})
Row(('https://stackoverflow.com/questions/13468933', 16062), {'url': 0, 'view_count': 1})


In [19]:
# This is where pandas comes in handy
results_df = pd.read_gbq(results)

ValueError: DataFrame constructor not properly called!